**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

Philomène Chagniot

In [2]:
# ! pip install sk-video
# # pip install ffmpeg-python
# import ffmpeg
! pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import skvideo

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense,Flatten
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

     |████████████████████████████████| 2.3MB 4.9MB/s 


Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


__Answer__ : 
The function `act` takes as input the state, and a boolean that indicates whether the function is used during training or not. If it is used for testing, the function act returns the policy learnt during the training, ie the "best action" given the state in input learnt by the agent. If the function `act` is used during training, the algorithm follows the policy learnt until now to select the action with probability $(1-\epsilon)$. With probability $\epsilon$, it uniformly selects an action at random among all possible actions. <br>


Thus, __if $\epsilon$ is too small, the agent only follows an "exploitation" strategy__ : it behaves in a greedy way, and select the actions that he estimated being the best, which may not be the case in reality. Thus, it is important to "explore" other actions, in order to avoid repeating actions that are actually suboptimal and thus to increase the cumulated rewards. However, __if the exploration parameter $\epsilon$ is too big, the agent never "exploits" what he has learnt__, and the cumulative reward will be similar as the one of a random agent, thus suboptimal. 


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat 
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0: ## go right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: ##go left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: ## go up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: ##go down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1 #time
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0 #once poison or cheese is eaten it can't be eaten again
        game_over = self.t > self.max_time
        # state : both the board and the position which may change at each step
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # the rat only knows this (limited visibility)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        
        # random initial position
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        
        # more bonus if higher temperature
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)
        
        # more malus if higher temperature
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        # if both malus and bonus --> only bonus
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging 
epochs_test=30 # set small when debugging 

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer__: The arrays position and board represent the current position of the rat, and the board configuration at a given moment. Both arrays have two dimensions (we are considering a game in 2D), and 
- $position[i, j]$ is equal to 1 if the rat is a the position $(i,j)$, 0 if not, and -1 if this position is outside of the island. 
- $board[i, j]$ is equal to 0.5 if there is cheese at the position $(i,j)$, 0 if there is nothing (or if the cheese or the poison has already been eaten), and -1 if there is poison.

The two arrays put together represent the current state of the game.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

__Answer__ : The random agent randomly selects one action among all possible actions (here according to a uniform distribution)

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Randomly select an action among the n_action actions possible
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix='',train=False):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # initialization
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
          
#             # Save as a mp4
            env.draw(prefix+str(e))

            # Update stats
            score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
                  .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [86]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/7.0. Average score (736.5)
Win/lose count 8.5/16.0. Average score (-65.5)
Win/lose count 6.0/22.0. Average score (-711.5)
Win/lose count 13.0/15.0. Average score (-718.875)
Win/lose count 5.5/13.0. Average score (-772.8)
Win/lose count 7.5/16.0. Average score (-840.1666666666666)
Win/lose count 14.5/12.0. Average score (-650.3571428571429)
Win/lose count 11.0/13.0. Average score (-577.9375)
Win/lose count 7.0/13.0. Average score (-585.3888888888889)
Win/lose count 9.0/17.0. Average score (-645.8)
Win/lose count 11.5/7.0. Average score (-554.5454545454545)
Win/lose count 13.0/11.0. Average score (-478.25)
Win/lose count 8.5/10.0. Average score (-452.7307692307692)
Win/lose count 11.0/17.0. Average score (-498.67857142857144)
Win/lose count 11.5/11.0. Average score (-461.96666666666664)
Win/lose count 5.5/5.0. Average score (-421.625)
Win/lose count 10.5/14.0. Average score (-426.3529411764706)
Win/lose count 10.0/12.0. Average score (-416.8888888888889)
Win/lose coun

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ 

__Question:__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

__Answer__ (I neglected the indices of the expectation function)
Let's show that $Q^{\pi}$ verifies the Bellman equation.
By splitting the sum between $t=0$ and $t \ge 1$ :
\begin{equation*}
Q^{\pi}(s,a) = E[ \sum_{t=0}^{\infty} \gamma^t r_{t}|s_0 = s, a_0 = a] = E[ r(s,a) + \sum_{t=1}^{\infty} \gamma^t r_{t}|s_0 = s, a_0 = a]
\end{equation*}

Then, we do a change of index, take $\gamma$ out of the expectation, and take $r(s,a)$ out using the conditionnig. 
\begin{equation*}
Q^{\pi}(s,a) =  r(s,a) + \gamma E[E[\sum_{t=0}^{\infty}[\gamma^t r_{t+1}|s_{1}=s',a_{1}=a']|s_0 = s, a_0 = a]
\end{equation*}
Then, we use the definition of the expectation, and the Markov property to remove $s_0 = s$ and $a_0 = a$ from the expectation. 
\begin{equation*}
Q^{\pi}(s,a) =  r(s,a) + \gamma \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )[E[\sum_{t=0}^{\infty}\gamma^t r_{t+1}|s_{1}=s',a_{1}=a']
\end{equation*}
Then, we use the definition of the Q-function :
\begin{equation*}
Q^{\pi}(s,a) =  r(s,a) + \gamma\sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )[Q^{\pi}(s',a')]
\end{equation*}
Thus, in fine we obtain : 
\begin{equation*}
Q^{\pi}(s,a) = E[ \sum_{t=0}^{\infty} \gamma^t r_{t}|s_0 = s, a_0 = a] = E[ r(s,a) + \sum_{t=1}^{\infty} \gamma^t r_{t}|s_0 = s, a_0 = a]
\end{equation*}


__Question :__
Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

__Answer__ 
By definition : 
\begin{equation*}
Q^{*}(s,a) = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}
Then we can use the Bellman equation (prooved just above) and get :
\begin{equation*}
Q^{*}(s,a) = \max_{\pi}[r(s,a) + \gamma \sum_{(s',a')}(p(s_{1}=s',a_{1}=a' |s_0 = s, a_0 = a )Q^{\pi}(s',a')]
\end{equation*}
Then, we use the fact that any policy $\pi$ can be split into a first action a' and a policy $\pi'$ for the remaining steps. 
\begin{equation*}
Q^{*}(s,a) = r(s,a) + \gamma \sum_{s'}(p(s_{1}=s'|s_0 = s, a_0 = a )\max_{(a',\pi')} Q^{\pi'}(s',a')
\end{equation*}
Then we split the max between $\pi'$ and $a'$.
\begin{equation*}
Q^{*}(s,a) = r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} \max_{\pi'}Q^{\pi'}(s',a')
\end{equation*}
Then we use the definition of $Q^{*}$
\begin{equation*}
Q^{*}(s,a) = r(s,a) + \gamma \sum(p(s_{t+1}=s' |s_t = s, a_t = a )\max_{a'} Q^{*}(s',a')
\end{equation*}
Thus, we get :
\begin{equation*}
Q^{*}(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
\end{equation*}

__Question :__
Finally, deduce that a plausible objective is:
\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

__Answer__ 
This is a plausible objective, because the loss is minimal for $Q^{*}$ (its value is 0 by the above equation), and strictly positive for any non-optimal policy ($\pi^{*}$ is not necessarily unique).

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)<self.max_memory:
            self.memory.append(m)

    def random_access(self):
        random_index=np.random.randint(0,len(self.memory))
        return self.memory[random_index]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):#mettre reseau de neurone ici (clement)
        d1,d2,d3 = s.shape
        pred = self.model.predict(s.reshape((1,d1,d2,d3)))
        return np.argmax(pred)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        # remembers only if self.memory is not already at the memory limit
        self.memory.remember(m=[s_, n_s_, a_, r_, game_over_])
        
        # initialize
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        d1,d2,d3=s_.shape #always the same shape
        
        for i in range(self.batch_size):
            # draw a sample
            s_sample, n_s_sample, a_sample, r_sample, game_over_sample = self.memory.random_access()
            
            # update the input state
            input_states[i,:,:,:] = s_sample
            
            # update the target q function
            target_q[i,:] = self.model.predict(s_sample.reshape(1,d1,d2,d3))


            if game_over_:
                target_q[i, a_sample] = r_sample
                
            else:
                target_q[i, a_sample] = r_sample + (self.discount*np.max(self.model.predict(n_s_sample.reshape((1,d1,d2,d3)))))
                 
                ######## FILL IN
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        #define the loss
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        # model.compile("sgd", "mse")
        # Use Adam optimizer instead
        model.compile("Adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model : fully connected
        # the input shape is corresponds to the neighborhood of 2 around the rate
        model = Sequential()
        model.add(Dense(40,activation ='relu', input_shape=(5,5,self.n_state)))
        model.add(Dense(20,activation ='relu'))
        model.add(Flatten())
        # output 
        model.add(Dense(4))
        # Use Adam optimizer instead
        model.compile(Adam(lr=lr), "mse") 
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32) ## try different values
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))













Epoch 000/030 | Loss 2.0752 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/030 | Loss 1.5819 | Win/lose count 7.5/13.0 (-5.5)
Epoch 002/030 | Loss 1.8955 | Win/lose count 19.0/18.0 (1.0)
Epoch 003/030 | Loss 1.9300 | Win/lose count 13.5/20.0 (-6.5)
Epoch 004/030 | Loss 1.5653 | Win/lose count 9.5/20.0 (-10.5)
Epoch 005/030 | Loss 1.4116 | Win/lose count 9.0/10.0 (-1.0)
Epoch 006/030 | Loss 1.5326 | Win/lose count 9.5/8.0 (1.5)
Epoch 007/030 | Loss 1.5177 | Win/lose count 10.5/16.0 (-5.5)
Epoch 008/030 | Loss 1.6663 | Win/lose count 11.0/9.0 (2.0)
Epoch 009/030 | Loss 1.5778 | Win/lose count 12.5/24.0 (-11.5)
Epoch 010/030 | Loss 1.4177 | Win/lose count 10.0/12.0 (-2.0)
Epoch 011/030 | Loss 1.6218 | Win/lose count 7.0/15.0 (-8.0)
Epoch 012/030 | Loss 1.6975 | Win/lose count 9.0/5.0 (4.0)
Epoch 013/030 | Loss 1.8070 | Win/lose count 11.0/13.0 (-2.0)
Epoch 014/030 | Loss 1.5585 | Win/lose count 16.0/15.0 (1.0)
Epoch 015/030 | Loss 1.6398 | Win/lose count 15.5/14.0 (1.5)
Epoch 016/0

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32) ## try different values
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))









Epoch 000/030 | Loss 1.8939 | Win/lose count 11.5/11.0 (0.5)
Epoch 001/030 | Loss 1.8663 | Win/lose count 11.0/9.0 (2.0)
Epoch 002/030 | Loss 1.5209 | Win/lose count 8.0/12.0 (-4.0)
Epoch 003/030 | Loss 1.7983 | Win/lose count 4.0/8.0 (-4.0)
Epoch 004/030 | Loss 1.8718 | Win/lose count 9.5/19.0 (-9.5)
Epoch 005/030 | Loss 1.7385 | Win/lose count 12.0/13.0 (-1.0)
Epoch 006/030 | Loss 1.5121 | Win/lose count 10.0/9.0 (1.0)
Epoch 007/030 | Loss 1.4845 | Win/lose count 9.0/16.0 (-7.0)
Epoch 008/030 | Loss 1.3565 | Win/lose count 14.5/15.0 (-0.5)
Epoch 009/030 | Loss 1.3540 | Win/lose count 12.0/20.0 (-8.0)
Epoch 010/030 | Loss 1.5387 | Win/lose count 9.5/12.0 (-2.5)
Epoch 011/030 | Loss 1.6704 | Win/lose count 12.5/13.0 (-0.5)
Epoch 012/030 | Loss 1.7233 | Win/lose count 13.5/24.0 (-10.5)
Epoch 013/030 | Loss 1.2503 | Win/lose count 17.0/18.0 (-1.0)
Epoch 014/030 | Loss 1.3967 | Win/lose count 16.5/24.0 (-7.5)
Epoch 015/030 | Loss 1.4531 | Win/lose count 12.0/12.0 (0.0)
Epoch 016/0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,kernel_size = 2,activation='relu', input_shape=(5,5,self.n_state)))
        model.add(Conv2D(25,kernel_size = 2,activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))

        # Use Adam optimizer instead
        model.compile(Adam(lr=lr), "mse")
        # model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 1.7019 | Win/lose count 4.5/4.0 (0.5)
Epoch 001/030 | Loss 1.9864 | Win/lose count 7.0/6.0 (1.0)
Epoch 002/030 | Loss 1.9297 | Win/lose count 7.5/9.0 (-1.5)
Epoch 003/030 | Loss 1.7759 | Win/lose count 6.5/13.0 (-6.5)
Epoch 004/030 | Loss 1.5165 | Win/lose count 14.5/13.0 (1.5)
Epoch 005/030 | Loss 1.5563 | Win/lose count 13.0/21.0 (-8.0)
Epoch 006/030 | Loss 1.5445 | Win/lose count 10.5/19.0 (-8.5)
Epoch 007/030 | Loss 1.6345 | Win/lose count 11.0/20.0 (-9.0)
Epoch 008/030 | Loss 1.7460 | Win/lose count 12.5/15.0 (-2.5)
Epoch 009/030 | Loss 1.4986 | Win/lose count 9.5/17.0 (-7.5)
Epoch 010/030 | Loss 1.3383 | Win/lose count 15.5/14.0 (1.5)
Epoch 011/030 | Loss 0.7972 | Win/lose count 12.0/21.0 (-9.0)
Epoch 012/030 | Loss 1.7824 | Win/lose count 8.5/20.0 (-11.5)
Epoch 013/030 | Loss 1.7267 | Win/lose count 11.5/15.0 (-3.5)
Epoch 014/030 | Loss 1.3613 | Win/lose count 11.5/14.0 (-2.5)
Epoch 015/030 | Loss 1.0408 | Win/lose count 5.5/7.0 (-1.5)
Epoch 016/030 | Loss 0

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 1.0400 | Win/lose count 4.0/6.0 (-2.0)
Epoch 001/030 | Loss 0.6398 | Win/lose count 8.0/8.0 (0.0)
Epoch 002/030 | Loss 1.4390 | Win/lose count 1.5/4.0 (-2.5)
Epoch 003/030 | Loss 1.2138 | Win/lose count 7.0/6.0 (1.0)
Epoch 004/030 | Loss 1.3267 | Win/lose count 5.5/15.0 (-9.5)
Epoch 005/030 | Loss 1.1149 | Win/lose count 4.5/15.0 (-10.5)
Epoch 006/030 | Loss 0.8604 | Win/lose count 9.0/9.0 (0.0)
Epoch 007/030 | Loss 0.7420 | Win/lose count 8.0/17.0 (-9.0)
Epoch 008/030 | Loss 0.6315 | Win/lose count 13.5/17.0 (-3.5)
Epoch 009/030 | Loss 0.7909 | Win/lose count 14.5/13.0 (1.5)
Epoch 010/030 | Loss 0.8127 | Win/lose count 9.0/13.0 (-4.0)
Epoch 011/030 | Loss 1.0493 | Win/lose count 14.0/14.0 (0.0)
Epoch 012/030 | Loss 0.6284 | Win/lose count 14.0/17.0 (-3.0)
Epoch 013/030 | Loss 0.9042 | Win/lose count 9.0/20.0 (-11.0)
Epoch 014/030 | Loss 0.8189 | Win/lose count 13.5/15.0 (-1.5)
Epoch 015/030 | Loss 0.6254 | Win/lose count 12.0/24.0 (-12.0)
Epoch 016/030 | Loss 1.39

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

__Answer__ : I implemented the CNN and the FC networks with a temperature of 0.1, 0.3 or 0.5 and ran both twice. The performance of the CNN is better than the performance of the FC for a medium temperature or high temperature. The more the temperature increases (from 0.1 to 0.3 and from 0.3 to 0.5), the better the performances of both networks are, since there is more bonus on the board. From the videos, one can observe that the rat doesn't explore much : it usually stays in the same "neighborhood" in a while, and thus visits the same cells many times since the rat doesn't 'remember' the cells which have already been visited. 

###### First trial to loop over values of temperature

In [0]:
# Loop over low, medium and high value of the temperature
for temp in [0.1,0.3,0.5]:
    env = Environment(grid_size=size, max_time=T,temperature=temp)

    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
    agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') #change to load agent_fc and not agent_cnn
    
    # Print results
    print('\n\n\nTest of the CNN with temperature',temp)
    test(agent_cnn,env,epochs_test,prefix='cnn_test')
    print('\nTest of the FC with temperature',temp)
    test(agent_fc,env,epochs_test,prefix='fc_test')




Test of the CNN with temperature 0.1
Win/lose count 0.5/2.0. Average score (-308.0)
Win/lose count 2.0/2.0. Average score (-150.75)
Win/lose count 0.5/3.0. Average score (-231.5)
Win/lose count 1.0/1.0. Average score (-156.0)
Win/lose count 1.5/1.0. Average score (-130.1)
Win/lose count 1.0/1.0. Average score (-128.91666666666666)
Win/lose count 1.5/3.0. Average score (-129.42857142857142)
Win/lose count 0/1.0. Average score (-137.625)
Win/lose count 1.0/0. Average score (-100.38888888888889)
Win/lose count 1.5/5.0. Average score (-124.8)
Win/lose count 1.0/0. Average score (-99.72727272727273)
Win/lose count 0.5/2.0. Average score (-114.5)
Win/lose count 1.0/4.0. Average score (-151.96153846153845)
Win/lose count 0.5/0. Average score (-134.07142857142858)
Win/lose count 0/1.0. Average score (-132.26666666666668)
Win/lose count 1.0/0. Average score (-111.625)
Win/lose count 1.5/2.0. Average score (-114.70588235294117)
Win/lose count 0.5/0. Average score (-105.63888888888889)
Win/los

###### Second trial to loop over values of temperature

In [16]:
# Low temperature
temp=0.1
env = Environment(grid_size=size, max_time=T,temperature=temp)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') #change to load agent_fc and not agent_cnn

# Print results
print('\n\n\nTest of the CNN with temperature',temp)
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC with temperature',temp)
test(agent_fc,env,epochs_test,prefix='fc_test')




Test of the CNN with temperature 0.1
Win/lose count 0.5/1.0. Average score (-77.0)
Win/lose count 0.5/1.0. Average score (-52.0)
Win/lose count 1.0/3.0. Average score (-199.0)
Win/lose count 0/4.0. Average score (-342.75)
Win/lose count 0.5/1.0. Average score (-287.9)
Win/lose count 1.0/0. Average score (-223.75)
Win/lose count 0/1.0. Average score (-213.35714285714286)
Win/lose count 2.0/1.0. Average score (-160.0625)
Win/lose count 1.5/1.0. Average score (-122.94444444444444)
Win/lose count 0/1.0. Average score (-124.85)
Win/lose count 0.5/3.0. Average score (-148.77272727272728)
Win/lose count 0.5/0. Average score (-128.125)
Win/lose count 1.0/1.0. Average score (-119.92307692307692)
Win/lose count 0.5/1.0. Average score (-118.92857142857143)
Win/lose count 1.0/0. Average score (-106.16666666666667)
Win/lose count 0.5/4.0. Average score (-131.5)
Win/lose count 0/2.0. Average score (-141.47058823529412)
Win/lose count 1.5/1.0. Average score (-132.94444444444446)
Win/lose count 2.5

In [15]:
# Medium temperature
temp=0.3
env = Environment(grid_size=size, max_time=T,temperature=temp)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') #change to load agent_fc and not agent_cnn

# Print results
print('\n\n\nTest of the CNN with temperature',temp)
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC with temperature',temp)
test(agent_fc,env,epochs_test,prefix='fc_test')




Test of the CNN with temperature 0.3
Win/lose count 1.5/4.0. Average score (-439.0)
Win/lose count 2.0/5.0. Average score (-397.75)
Win/lose count 2.0/3.0. Average score (-287.6666666666667)
Win/lose count 2.0/4.0. Average score (-258.375)
Win/lose count 4.0/5.0. Average score (-201.7)
Win/lose count 2.5/2.0. Average score (-157.08333333333334)
Win/lose count 2.5/4.0. Average score (-144.07142857142858)
Win/lose count 3.5/3.0. Average score (-134.4375)
Win/lose count 2.0/1.0. Average score (-104.16666666666667)
Win/lose count 3.5/7.0. Average score (-152.65)
Win/lose count 1.5/3.0. Average score (-160.95454545454547)
Win/lose count 4.5/3.0. Average score (-123.5)
Win/lose count 3.0/3.0. Average score (-103.07692307692308)
Win/lose count 2.0/3.0. Average score (-125.0)
Win/lose count 1.5/1.0. Average score (-117.03333333333333)
Win/lose count 4.0/2.0. Average score (-75.25)
Win/lose count 2.5/2.0. Average score (-75.05882352941177)
Win/lose count 2.0/2.0. Average score (-66.111111111

In [16]:
# High temperature
temp=0.5
env = Environment(grid_size=size, max_time=T,temperature=temp)

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json') #change to load agent_fc and not agent_cnn

# Print results
print('\n\n\nTest of the CNN with temperature',temp)
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('\nTest of the FC with temperature',temp)
test(agent_fc,env,epochs_test,prefix='fc_test')




Test of the CNN with temperature 0.5
Win/lose count 5.0/5.0. Average score (17.5)
Win/lose count 4.5/4.0. Average score (-44.75)
Win/lose count 4.0/4.0. Average score (-9.666666666666666)
Win/lose count 4.5/2.0. Average score (60.875)
Win/lose count 6.0/6.0. Average score (81.8)
Win/lose count 5.5/4.0. Average score (88.66666666666667)
Win/lose count 3.0/3.0. Average score (65.57142857142857)
Win/lose count 3.5/0. Average score (107.1875)
Win/lose count 6.5/1.0. Average score (160.05555555555554)
Win/lose count 3.0/5.0. Average score (103.1)
Win/lose count 3.5/3.0. Average score (112.95454545454545)
Win/lose count 4.0/2.0. Average score (122.04166666666667)
Win/lose count 3.0/5.0. Average score (92.23076923076923)
Win/lose count 5.5/1.0. Average score (127.96428571428571)
Win/lose count 4.0/1.0. Average score (146.76666666666668)
Win/lose count 5.5/3.0. Average score (160.84375)
Win/lose count 5.0/6.0. Average score (141.85294117647058)
Win/lose count 2.5/3.0. Average score (127.083

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix='',train=True):
    ''' The function is defined as train, but includes exploration'''
      # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        # Increase decreasing epsilon-greedy exploration
        agent.set_epsilon(agent.epsilon*0.9)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model_explo.h5',name_model=prefix+'model_explo.json')
        
class EnvironmentExploring(object):
    ''' this class is equivalent to the Environment class, except that a malus for already visited positions is added'''
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        # add malus position
        self.malus_position=np.zeros((grid_size,grid_size))

        # coordinate of the cat 
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0: ## go right
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1: ##go left
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2: ## go up
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3: ##go down
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1 #time

        #update the malus with new position visited :
        # during the training step, we integrate a malus of the position has already been visited
        # during the test step, the malus_position matrix will only contain zeros
        if train==True :
            self.malus_position[self.x,self.y]=0.1

        # the reward is the sum of the malus for the position and of the board malus/bonus
        reward = self.board[self.x, self.y]-self.malus_position[self.x,self.y]

        self.board[self.x, self.y] = 0 #once poison or cheese is eaten it can't be eaten again
        game_over = self.t > self.max_time
        # state : 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # the rat only knows this (limited visibility)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        
        # random initial position
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        
        # more bonus if higher temperature
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)
        
        # more malus if higher temperature
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        # if both malus and bonus --> only bonus
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # initialize the malus for the position to 0 :
        self.malus_position=np.zeros((self.grid_size,self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [40]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/030 | Loss 1.7591 | Win/lose count 8.000000000000002/28.10000000000007 (-20.100000000000065)
Epoch 001/030 | Loss 0.6879 | Win/lose count 10.400000000000004/29.500000000000092 (-19.100000000000087)
Epoch 002/030 | Loss 0.0844 | Win/lose count 9.200000000000003/30.800000000000118 (-21.600000000000115)
Epoch 003/030 | Loss 0.2799 | Win/lose count 8.800000000000002/32.900000000000134 (-24.10000000000013)
Epoch 004/030 | Loss 1.2842 | Win/lose count 7.200000000000002/30.300000000000068 (-23.100000000000065)
Epoch 005/030 | Loss 1.0623 | Win/lose count 8.800000000000002/29.90000000000012 (-21.100000000000115)
Epoch 006/030 | Loss 1.2391 | Win/lose count 6.800000000000002/32.40000000000012 (-25.60000000000012)
Epoch 007/030 | Loss 1.2479 | Win/lose count 11.600000000000005/29.200000000000113 (-17.600000000000108)
Epoch 008/030 | Loss 1.2189 | Win/lose count 5.2/30.800000000000097 (-25.600000000000097)
Epoch 009/030 | Loss 1.3139 | Win/lose count 8.000000000000002/28.10000000000007 

In [43]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0.5/0. Average score (100.5)
Win/lose count 1.0/1.0. Average score (50.25)
Win/lose count 1.0/2.0. Average score (-32.0)
Win/lose count 1.0/0. Average score (26.0)
Win/lose count 1.5/2.0. Average score (0.4)
Win/lose count 0/0. Average score (0.3333333333333333)
Win/lose count 0.5/2.0. Average score (-35.0)
Win/lose count 1.0/1.0. Average score (-39.9375)
Win/lose count 1.5/1.0. Average score (-24.944444444444443)
Win/lose count 3.0/1.0. Average score (15.1)
Win/lose count 0.5/4.0. Average score (-49.0)
Win/lose count 3.5/0. Average score (12.166666666666666)
Win/lose count 1.5/1.0. Average score (27.192307692307693)
Win/lose count 2.0/1.0. Average score (39.714285714285715)
Win/lose count 2.0/0. Average score (63.3)
Win/lose count 0.5/1.0. Average score (53.1875)
Win/lose count 1.0/0. Average score (58.64705882352941)
Win/lose count 0/0. Average score (55.388888888888886)
Win/lose count 2.0/1.0. Average score (62.94736842105263)
Win/lose count 0/0. Average score (59.8)


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***